In [1]:
import pandas as pd
import numpy as np

## INPUT DATA 

In [154]:
types_of_loadings = ['concentrated','moment','C_dist','L_dist']
joint_data = pd.DataFrame(np.array([[1,0],[2,15],[3,30]]),columns=['joint','x'])
suport_data = pd.DataFrame(np.array([[1,1,0],[3,1,0]]),columns = ['suport','y','r'])
materialp_data = pd.DataFrame(np.array([[1,200*(10**6)]]),columns=['type','E'])
crossec_data = pd.DataFrame(np.array([[1,0.163]]),columns=['type','inertia'])
member_data = pd.DataFrame(np.array([[1,1,2,1,1],[2,2,3,1,1]]),columns=['member','start','end','material_type','inertia_type'])
jointload_data = pd.DataFrame(np.array([[1,0,0]]),columns = ['joint','y_load','m_load'])
memberload_data = pd.DataFrame(np.array([[1,'C_dist',194,(0,5)],[2,'C_dist',194,(5,0)]]),columns = ['member','loadtype','w','position'])
names = zip(['joint_data','suport_data','materialp_data','crossec_data','member_data','jointload_data','member_loadata'],
             [joint_data,suport_data,materialp_data,crossec_data,member_data,jointload_data,memberload_data])

for i,a in names:
    print(i,'\n',a)
    print('\n')

<ipython-input-154-5b6dd4cc75c7>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  memberload_data = pd.DataFrame(np.array([[1,'C_dist',194,(0,5)],[2,'C_dist',194,(5,0)],


joint_data 
    joint   x
0      1   0
1      2  15
2      3  30


suport_data 
    suport  y  r
0       1  1  0
1       3  1  0


materialp_data 
    type          E
0     1  200000000


crossec_data 
    type  inertia
0   1.0    0.163


member_data 
    member  start  end  material_type  inertia_type
0       1      1    2              1             1
1       2      2    3              1             1


jointload_data 
    joint  y_load  m_load
0      1       0       0


member_loadata 
   member      loadtype    w position
0      1        C_dist  194   (0, 5)
1      2        C_dist  194   (5, 0)
2      1  concentrated  294     12.5
3      1  concentrated  294   14.167
4      2  concentrated  294   0.8335
5      2  concentrated  294      2.5




## SETTING PARAMETERS

In [155]:
ncjt = 2
nj = len(joint_data)
ns = len(suport_data)
rc = (suport_data[['y','r']] == 1).sum().sum()
ndof = (ncjt*nj)-rc

## ESTRUCTURE COORDINATE NUMBERS

In [156]:
def structure_numbers():
    k = ndof
    j=0
    str_number = np.zeros(nj*2).reshape(nj*2,1)
    for i in range(1,nj+1):
        if any(suport_data.suport==i):
            row = suport_data[suport_data.suport == i]
            if row.y.values == 1:
                k+=1
                str_number[(ncjt*i)-2]=k
            else:
                j+=1
                str_number[(ncjt*i)-2]=j
            if row.r.values == 1:
                k+=1
                str_number[(ncjt*i)-1]=k
            else:
                j+=1
                str_number[(ncjt*i)-1]=j
        else:
            j+=1
            str_number[(ncjt*i)-2]=j
            j+=1
            str_number[(ncjt*i)-1]=j
    return str_number.astype('int64')

str_number = structure_numbers()
str_number

array([[5],
       [1],
       [2],
       [3],
       [6],
       [4]], dtype=int64)

## MEMBER PARAMETERS AND STIFFNESS MATRIX

In [157]:
def member_parameters(member_number):
    info = member_data[member_data.member==member_number]
    start = info.start.values.item()
    end = info.end.values.item()
    E = materialp_data[materialp_data.type==info.material_type.values.item()].E.values.item()
    inertia = crossec_data[crossec_data.type==info.inertia_type.values.item()].inertia.values.item()
    x_b = joint_data[joint_data.joint == start].x.item()
    x_e = joint_data[joint_data.joint == end].x.item()
    l = x_e-x_b
    membercode_number=np.array([str_number[(ncjt*start)-2].item(),str_number[(ncjt*start)-1].item()
                       ,str_number[(ncjt*end)-2].item(),str_number[(ncjt*end)-1].item()]).reshape(2*ncjt,1)
    return (E,inertia,l,membercode_number)

    
def stiffness_matrix(member_number):
    E,inertia,l,membercode_number = member_parameters(member_number)
    stiffness = E*inertia/l**3
    k = stiffness*np.array([12,6*l,-12,6*l,6*l,4*l**2,-6*l,2*l**2,
                                  -12,-6*l,12,-6*l,6*l,2*l**2,-6*l,4*l**2]).reshape(4,4)
    return (k,membercode_number)

## ESTRUCTURE STIFFNESS MATRIX

In [158]:
def structure_stiffness_matrix():
    members = len(member_data)
    S = np.zeros(ndof*ndof).reshape(ndof,ndof)
    for member in range (1,members+1):
        k,membercode_number = stiffness_matrix(member)

        for i in range(1,(2*ncjt)+1):
            n1 = membercode_number[i-1]
            if n1 <= ndof:
                for j in range(1,(2*ncjt)+1):
                    n2 = membercode_number[j-1]
                    if n2 <= ndof:
                        S[n1-1,n2-1] = S[n1-1,n2-1] + k[i-1,j-1]
                    else:
                        continue
            else:
                continue
    return S
structure_stiffness_matrix()

array([[ 8693333.33333333,  -869333.33333333,  4346666.66666667,
               0.        ],
       [ -869333.33333333,   231822.22222222,        0.        ,
          869333.33333333],
       [ 4346666.66666667,        0.        , 17386666.66666667,
         4346666.66666667],
       [       0.        ,   869333.33333333,  4346666.66666667,
         8693333.33333333]])

## JOINT LOAD VECTOR

In [159]:
def joint_load_vector():
    p = np.zeros(ndof).reshape(ndof,1)
    rows = len(jointload_data)
    for i in range(rows):
        loaded_joint = jointload_data.iloc[i].joint.item()
        Ystr_number = str_number[(2*loaded_joint)-2].item()
        Rstr_number = str_number[(2*loaded_joint)-1].item()
        if Ystr_number <= ndof:
            p[Ystr_number-1] = jointload_data.iloc[i].y_load.item()
        else:
            None

        
        if Rstr_number <= ndof:
            p[Rstr_number-1] = jointload_data.iloc[i].m_load.item()
        else:
            None
    return p

joint_load_vector()

array([[0.],
       [0.],
       [0.],
       [0.]])

## MEMBER AND STRUCTURE EQUIVALENT JOINT LOAD VECTOR

In [160]:
def member_equivalent_joint_loadvector(member):
    info = memberload_data[memberload_data.member == member]
    Qf= np.zeros(2*ncjt).reshape(2*ncjt,1)
    for i in range(len(info)):
        lp = info.iloc[i].position
        load_type = info.iloc[i].loadtype
        w = info.iloc[i].w
        _,_,l,membercode_number = member_parameters(member)

        if load_type == types_of_loadings[0]:
            v1 = ((w/l**3)*(l-lp)**2)*((3*lp)+(l-lp))
            m1 = (w/l**2)*(lp*(l-lp)**2)
            v2 = ((w/l**3)*lp**2)*(lp+(3*(l-lp)))
            m2 = -1*(w/l**2)*((lp**2)*(l-lp))
            Qf = Qf + np.array([[v1],[m1],[v2],[m2]])

        elif load_type == types_of_loadings[1]:
            v1 = (-1*6*w/l**3)*(lp*(l-lp))
            m1 = ((w/l**2)*(l-lp))*((l-lp)-(2*lp))
            v2 = (6*w/l**3)*(lp*(l-lp))
            m2 = ((w/l**2)*lp)*(lp-(2*(l-lp)))
            Qf = Qf + np.array([[v1],[m1],[v2],[m2]])

        elif load_type == types_of_loadings[2]:
            v1 = (w*l/2)*(1-((lp[0]/l**4)*((2*l**3)-((2*lp[0]**2)*l)+lp[0]**3))-(((lp[1]**3)/l**4)*((2*l)-lp[1])))
            m1 = ((w*l**2)/12)*(1-(((lp[0]**2)/(l**4))*(6*l**2-8*lp[0]*l+3*lp[0]**2))-(((lp[1]**3)/(l**4))*(4*l-3*lp[1])))                                                                                       
            v2 = (w*l/2)*(1-(((lp[0]**3)/(l**4))*(2*l-lp[0]))-((lp[1]/l**4)*(2*l**3-((2*lp[1]**2)*l)+lp[1]**3)))
            m2 = (-1*(w*l**2)/12)*(1-(((lp[1]**2)/(l**4))*(6*l**2-8*lp[1]*l+3*lp[1]**2))-(((lp[0]**3)/(l**4))*(4*l-3*lp[0])))
            Qf = Qf + np.array([[v1],[m1],[v2],[m2]])

        elif load_type == types_of_loadings[3]:
            a = (7*l+8*lp[0])-(((lp[1]*(3*l+2*lp[0]))/(l-lp[0]))*(1+((lp[1])/(l-lp[0]))+((lp[1]**2)/(l-lp[0])**2)))+((2*lp[1]**4)/(l-lp[0])**3) 
            b = ((3*l+2*lp[0])*(1+((lp[1])/(l-lp[0]))+((lp[1]**2)/(l-lp[0])**2)))-(((lp[1]**3)/(l-lp[0])**2)*(2+((15*l-8*lp[1])/(l-lp[0]))))
            c = (3*l+12*lp[0])-(((lp[1]*(2*l+3*lp[0]))/(l-lp[0]))*(1+((lp[1])/(l-lp[0]))+((lp[1]**2)/(l-lp[0])**2)))+((3*lp[1]**4)/(l-lp[0])**3)
            d = ((2*l+3*lp[0])*(1+((lp[1])/(l-lp[0]))+((lp[1]**2)/(l-lp[0])**2)))-(((3*lp[1]**3)/(l-lp[0])**2)*(1+((5*l-4*lp[1])/(l-lp[0]))))
            v1 = a*((w[0]*(l-lp[0])**3)/(20*l**3))+b*((w[1]*(l-lp[0])**3)/(20*l**3))
            m1 = c*((w[0]*(l-lp[0])**3)/(60*l**2))+d*((w[1]*(l-lp[0])**3)/(60*l**2))
            v2 = (((w[0]+w[1])/2)*(l-lp[0]-lp[1]))-v1
            m2 = (((l-lp[0]-lp[1])/6)*((w[0]*(-2*l+2*lp[0]-lp[1]))-(w[1]*(l-lp[0]+2*lp[1]))))+(v1*l)-m1
            Qf = Qf + np.array([[v1],[m1],[v2],[m2]])
        
        else:
            print('bad inputs in: '.format(load_type))
            
        
    return Qf


def structure_equivalent_joint_loadvector():
    pf = np.zeros(ndof).reshape(ndof,1)
    for m in set(memberload_data.member.values):
        _,_,l,membercode_number = member_parameters(m)
        Qf= member_equivalent_joint_loadvector(m)
        for i,a in zip(membercode_number.reshape(-1,),list(range(len(Qf)))):
            if i <= ndof:
                pf[i-1] = pf[i-1] + Qf[a].item()
            else:
                None
    return pf

structure_equivalent_joint_loadvector()

array([[ 3.34826163e+03],
       [ 2.27683230e+03],
       [ 1.15697439e-01],
       [-3.34827660e+03]])

## JOINT DISPLACEMENTS

In [161]:
displacements = np.linalg.inv(structure_stiffness_matrix()).dot(joint_load_vector() - structure_equivalent_joint_loadvector())
displacements

array([[-5.46919238e-03],
       [-5.08404431e-02],
       [-9.44653907e-09],
       [ 5.46920355e-03]])

## REACTIONS

In [162]:
members = len(member_data)
reactions = np.zeros(rc).reshape(rc,1)
for member in range(1,members+1):
    _,_,_,membercode_number = member_parameters(member)
#     T = transformation_matrix(member)
    k,_ = stiffness_matrix(member) 
    memb_disp = np.zeros(2*ncjt).reshape(2*ncjt,1)
    for i in range(2*ncjt):
        n = membercode_number[i].item() 
        if n <= ndof:
            memb_disp[i]=displacements[n-1]
        else:
            None
    memb_forces = k.dot(memb_disp) + member_equivalent_joint_loadvector(member)
    for i in range(2*ncjt):
        n = membercode_number[i].item()
        if n > ndof:
            reactions[n-ndof-1]=reactions[n-ndof-1]+memb_forces[i]
        else:
            None
reactions

array([[2527.9951],
       [2528.0049]])

In [163]:
def joint_displacements():
    jt_displacements = np.zeros(ncjt*nj).reshape(ncjt*nj,1)

    for i in range(ncjt*nj):
        n= str_number[i]
        if n <= ndof:
            jt_displacements[i] = displacements[n-1]
        else:
            jt_displacements[i] = 0
    jt_displacements.reshape(nj,-1)
    joint_no = joint_data.joint.values
    final = pd.DataFrame(jt_displacements.reshape(nj,-1),columns=['y_disp','rot'],index =joint_no )
    final.index.name = 'joint.no'
    return (jt_displacements,final)


def suport_reactions():
    sup_reactions = np.zeros(ncjt*ns).reshape(ncjt*ns,1)
    j = 0        
    for i in suport_data.suport:
        j+=ncjt
        n1 = str_number[ncjt*i-2].item()
        n2 = str_number[ncjt*i-1].item()
        if n1> ndof:
            sup_reactions[j-2] = reactions[n1-ndof-1]
        else:
            None
        if n2 > ndof:
            sup_reactions[j-1] = reactions[n2-ndof-1]
        else:
            None
    suport_joints = suport_data.suport.values
    final = pd.DataFrame(sup_reactions.reshape(ns,-1),columns=['y_reac','moment'], index=suport_joints)
    final.index.name = 'joint.no'
    return final


def internal_forces():
    index = pd.MultiIndex(levels=[[],[]],codes=[[],[]],names=[u'member', u'joint'])
    int_forces = pd.DataFrame(index=index , columns=['shear','moment'])
    jd,_ = joint_displacements()
    for member in member_data.member.values:
        start = member_data[member_data.member == member].start.item()
        end = member_data[member_data.member == member].end.item()
        k,_ = stiffness_matrix(member)
        Qf = member_equivalent_joint_loadvector(member)
        disp = np.array([jd[(ncjt*start)-2],jd[(ncjt*start)-1],
                                jd[(ncjt*end)-2],jd[(ncjt*end)-1]])
        forces = np.dot(k,disp)+Qf
        forces = forces.reshape(-1,ncjt)
        int_forces.loc[(member,start),:] = forces[0]
        int_forces.loc[(member,end),:] = forces[1] 
    
    return int_forces


print(joint_displacements()[1],'\n','\n',suport_reactions(),'\n','\n',internal_forces())

           y_disp           rot
joint.no                       
1         0.00000 -5.469192e-03
2        -0.05084 -9.446539e-09
3         0.00000  5.469204e-03 
 
              y_reac  moment
joint.no                   
1         2527.9951     0.0
3         2528.0049     0.0 
 
                shear       moment
member joint                     
1      1        2528 -1.31877e-11
       2      0.0049        17540
2      2     -0.0049       -17540
       3        2528 -1.50067e-11


In [133]:
internal_forces()

shear       moment
member joint                      
1      1         2348 -8.18545e-12
       2      0.00465        16390
2      2     -0.00465       -16390
       3         2348  1.81899e-12